In [0]:
from pyspark.sql.functions import col
### PATHS ###
ADLS_SILVER = "abfss://citibike@databricksjm.dfs.core.windows.net/silver"
ADLS_GOLD = "abfss://citibike@databricksjm.dfs.core.windows.net/gold"
ADLS_TEST = "abfss://citibike@databricksjm.dfs.core.windows.net/test"

### AVAILABLE YEARS ###
years = []
for silver_dir in dbutils.fs.ls(ADLS_SILVER):
    dirname = silver_dir.name
    if dirname.startswith("_"):
        continue
    idx = dirname.index("=") + 1
    year_dir = dirname[idx:idx+4]
    years.append(year_dir)
years.sort()

### WIDGET ###
dbutils.widgets.multiselect(
    name="years",
    defaultValue=years[-1],
    choices=years,
    label="Select years"
)
YEARS_RAW = dbutils.widgets.get("years")
YEARS_STR = [y.strip() for y in YEARS_RAW.split(",") if y.strip()]  # ['2019','2020',...]

if not YEARS_STR:
    raise ValueError("No years selected")

### CREATE DF ###
df = (
    spark.read
    .format("delta")
    .load(ADLS_SILVER)
    .where(col("year").isin([int(y) for y in YEARS_STR]))
)

In [0]:
display(df)